In [224]:
import cv2
import numpy as np

image = cv2.imread('TwoShoes.jpg')

image = cv2.resize(image, (800, 600))

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

blurred = cv2.GaussianBlur(gray, (7, 7), 0)  

edges = cv2.Canny(blurred, 50,180)  

kernel = np.ones((5, 5), np.uint8)
closed_edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel, iterations=1)

contours, _ = cv2.findContours(closed_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

filtered_contours = []
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    aspect_ratio = w / float(h)
    area = cv2.contourArea(contour)
    if 1.5 < aspect_ratio < 4.5 and area > 3000:
        filtered_contours.append(contour)
filtered_contours = sorted(filtered_contours, key=cv2.contourArea, reverse=True)[:2]

# Check if two contours are found
if len(filtered_contours) == 2:
    contour1 = filtered_contours[0]
    contour2 = filtered_contours[1]
    
    # Retain more of the shoe's shape details
    epsilon1 = 0.0001 * cv2.arcLength(contour1, True)  
    epsilon2 = 0.0001 * cv2.arcLength(contour2, True)
    
    contour1 = cv2.approxPolyDP(contour1, epsilon1, True)
    contour2 = cv2.approxPolyDP(contour2, epsilon2, True)
    
    shape_similarity = cv2.matchShapes(contour1, contour2, cv2.CONTOURS_MATCH_I1, 0.0)
    print(f"Shape similarity: {shape_similarity}")

    if shape_similarity < 0.1:
        print("The shoes are very similar in shape.")
    elif 0.1 <= shape_similarity <= 0.5:
        print("The shoes have a moderate similarity in shape.")
    else:
        print("The shoes are not very similar in shape.")
 
 
    area1 = cv2.contourArea(contour1)
    area2 = cv2.contourArea(contour2)
    size_difference = abs(area1 - area2)

    print(f"Area of Red Shoe: {area1}")
    print(f"Area of Blue Shoe: {area2}")
    if area1 > area2:
        percentage_diff = ((area1 - area2) / area2) * 100
        print(f"Red Shoe is {percentage_diff:.2f}% bigger than Blue Shoe.")
        print("Meaning the red shoe is the newer shoe, and the blue shoe is the worn shoe.")
    elif area2 > area1:
        percentage_diff = ((area2 - area1) / area1) * 100
        print(f"Blue Shoe is {percentage_diff:.2f}% bigger than Red Shoe.")
        print("Meaning the blue shoe is the newer shoe, and the red shoe is the worn shoe.")
    else:
        print("Both shoes are the same size.")
    
    cv2.drawContours(image, [contour1], -1, (0, 0, 255), 2)  # Red
    cv2.drawContours(image, [contour2], -1, (255, 0, 0), 2)  # Blue

    cv2.imshow('Shoes with Contours', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    #Overlay two shoes to see differences

    overlay_canvas = np.zeros((600, 800, 3)) #3 means 3 channels -> RGB

    x1, y1, w1, h1 = cv2.boundingRect(contour1)
    x2, y2, w2, h2 = cv2.boundingRect(contour2)

    contour1_shifted = contour1 - [x1 - 100, y1 - 200]

    contour2_shifted = contour2 - [x2 - 100, y2 - 200]

    cv2.drawContours(overlay_canvas, [contour1_shifted], -1, (0, 0, 255), 1)  # Red for contour1
    cv2.drawContours(overlay_canvas, [contour2_shifted], -1, (255, 0, 0), 1)  # Blue for contour2

    # Show the overlaid contours for comparison
    cv2.imshow('Overlay Contour Comparison', overlay_canvas)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

else:
    print("Could not detect two shoes in the image.")


Shape similarity: 0.40004189635283555
The shoes have a moderate similarity in shape.
Area of Red Shoe: 68162.0
Area of Blue Shoe: 62737.5
Red Shoe is 8.65% bigger than Blue Shoe.
Meaning the red shoe is the newer shoe, and the blue shoe is the worn shoe.
